### Assignment-2 
"Create a simple User Management System where users can:
> Register by providing a name, email, address and password.
> Login using email and password.
> once logged in, they can see Profile(user details).
All user data should be in the following format:
name, email, address, password"


In [ ]:
pip install Flask
from flask import Flask, render_template, request, redirect, session
import sqlite3

app = Flask(__name__)
app.secret_key = 'your_secret_key'

# Initialize Database
def init_db():
    conn = sqlite3.connect('users.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT,
            email TEXT UNIQUE,
            address TEXT,
            password TEXT
        )
    ''')
    conn.commit()
    conn.close()

init_db()

# Register Route
@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        name = request.form['name']
        email = request.form['email']
        address = request.form['address']
        password = request.form['password']

        conn = sqlite3.connect('users.db')
        cursor = conn.cursor()
        cursor.execute('INSERT INTO users (name, email, address, password) VALUES (?, ?, ?, ?)', (name, email, address, password))
        conn.commit()
        conn.close()

        return redirect('/login')
    return render_template('register.html')

# Login Route
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        email = request.form['email']
        password = request.form['password']

        conn = sqlite3.connect('users.db')
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM users WHERE email=? AND password=?', (email, password))
        user = cursor.fetchone()
        conn.close()

        if user:
            session['user'] = user[1]  # Store the username in session
            return redirect('/profile')

    return render_template('login.html')

# Profile Route
@app.route('/profile')
def profile():
    if 'user' in session:
        return f"Welcome, {session['user']}! Here is your profile."
    return redirect('/login')

# Logout Route
@app.route('/logout')
def logout():
    session.pop('user', None)
    return redirect('/login')

if __name__ == '__main__':
    app.run(debug=True)